In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
from math import isnan
import datetime as dt
from datetime import datetime
from datetime import timedelta

In [2]:
file = 'cleanDataFinal-yifeng.csv'
myData = pd.read_csv(file)

# Delete unrelated attribute

In [3]:
myData.head(5)

,Unnamed: 0,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,0,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,1,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,2,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,3,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,4,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [4]:
del_attribute = 'Unnamed: 0'

In [5]:
myData.drop([del_attribute],axis = 1, inplace = True)

In [6]:
myData.head(5)

,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [7]:
def str_to_datetime (x):
    return datetime.strptime(x, '%Y-%m-%d')
myData['Date'] = myData['Date'].apply(str_to_datetime)

In [8]:
myData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96366 entries, 0 to 96365
Data columns (total 9 columns):
Cardnum              96366 non-null int64
Date                 96366 non-null datetime64[ns]
Merchnum             96366 non-null object
Merch description    96366 non-null object
Merch state          96366 non-null object
Merch zip            96366 non-null float64
Transtype            96366 non-null object
Amount               96366 non-null float64
Fraud                96366 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 6.6+ MB


In [9]:
myData.head(3)

,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0


# Begin Feature Engineering

In [10]:
data_sorted = myData.copy()

In [11]:
data_sorted.shape

(96366, 9)

In [12]:
%%time
#Cardnum, Merchnum
for groupbyvar in ['Cardnum', 'Merchnum']:
    data_sorted = data_sorted.sort_values(by = [groupbyvar, 'Date'])
    data_sorted_index = data_sorted.set_index('Date')
    for agg in ['mean', 'max', 'median', 'sum', 'count']: 
        for days in ['1d', '3d', '7d', '14d', '30d']:
            data_sorted[agg + '_' + groupbyvar + "_" + days]= \
                                getattr(data_sorted_index.groupby(groupbyvar)['Amount'].rolling(days),agg)().values
            data_sorted['Actual/' + agg + "_" + groupbyvar + "_" + days] = \
                                data_sorted['Amount']/data_sorted[agg + '_' + groupbyvar + "_" + days]

CPU times: user 1min 37s, sys: 907 ms, total: 1min 38s
Wall time: 1min 37s


In [13]:
data_sorted.shape

(96366, 109)

In [14]:
%%time
#Cardnum and Merchnum, Cardnum and Zip, Cardnum and state
for groupbyvar in ['Merchnum', 'Merch zip', 'Merch state']:
    data_sorted = data_sorted.sort_values(by = ['Cardnum',groupbyvar, 'Date'])
    data_sorted_index = data_sorted.set_index('Date')
    for agg in ['mean', 'max', 'median', 'sum', 'count']: 
        for days in ['1d', '3d', '7d', '14d', '30d']:
            data_sorted[agg + '_' + "Cardnum_" + groupbyvar + "_" + days] = getattr(data_sorted_index.groupby(['Cardnum',groupbyvar])['Amount'].rolling(days),agg)().values
            data_sorted['Actual/' + agg + "_" + "Cardnum_" + groupbyvar + "_" + days] = data_sorted['Amount']/data_sorted[agg + '_' + "Cardnum_" + groupbyvar + "_" + days]            


CPU times: user 11min 9s, sys: 18.1 s, total: 11min 27s
Wall time: 11min 18s


In [15]:
data_sorted.shape

(96366, 259)

In [16]:
%%time
#Create days since variables
for groupbyvar in [['Cardnum'], ['Merchnum'], ['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    sortCols = groupbyvar[:]
    sortCols.append('Date')
    data_sorted = data_sorted.sort_values(by = sortCols)
    if len(groupbyvar) == 1:
        data_sorted['Days_since_per_' + groupbyvar[0]] = \
                                data_sorted.groupby(groupbyvar)['Date'].\
                                apply(lambda x: (x - x.shift(1)).astype('timedelta64[D]')).fillna(365).values 
    else:
        data_sorted['Days_since_per_Cardnum_' + groupbyvar[1]] = \
                                data_sorted.groupby(groupbyvar)['Date'].\
                                apply(lambda x: (x -x.shift(1)).astype('timedelta64[D]')).fillna(365).values 


CPU times: user 1min, sys: 2.71 s, total: 1min 3s
Wall time: 57.3 s


In [17]:
data_sorted.shape

(96366, 264)

In [18]:
%%time
#Velocity change Variables
groupbyvar_numerator = ['Cardnum', 'Merchnum']
groupbyvar_denominator = ['Cardnum', 'Merchnum']

days_numerator = ['1d']
days_denominator = ['3d', '7d', '14d', '30d']

agg__numerator = ['count', 'mean']
agg__denominator = ['count', 'mean']

for a in groupbyvar_numerator:
    for b in groupbyvar_denominator:
        for c in days_numerator:
            for d in days_denominator:
                for e in agg__numerator:
                    for f in agg__denominator:
                        data_sorted[a + '_' + c + '_' + e + '_' + b + '_' + d + '_' + f] = \
                            data_sorted[e + '_' + a + "_" + c] / \
                            data_sorted[f + '_' + b + "_" + d]
                            

CPU times: user 87.8 ms, sys: 27.6 ms, total: 115 ms
Wall time: 79.1 ms


In [19]:
data_sorted.shape

(96366, 328)

In [20]:
%%time
#Velocity change Variables
groupbyvar_numerator = ['Cardnum', 'Merchnum']
groupbyvar_denominator = ['Cardnum', 'Merchnum']

days_numerator = ['1d']
days_denominator = ['3d', '7d', '14d', '30d']

agg__numerator = ['count', 'mean']
agg__denominator = ['count']

for a in groupbyvar_numerator:
    for b in groupbyvar_denominator:
        for c in days_numerator:
            for d in days_denominator:
                for e in agg__numerator:
                    for f in agg__denominator:
                        temp = d[0:len(d)-1]
                        data_sorted[a + '_' + c + '_' + e + '_' + b + '_' + d + '_' + f + '_Ave'] = \
                            data_sorted[e + '_' + a + "_" + c] / \
                            (data_sorted[f + '_' + b + "_" + d] / float(temp)  )

CPU times: user 248 ms, sys: 240 ms, total: 488 ms
Wall time: 411 ms


In [21]:
data_sorted.shape

(96366, 360)

In [22]:
data_sorted.head(5)

,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,mean_Cardnum_1d,...,Merchnum_1d_count_Cardnum_30d_count_Ave,Merchnum_1d_mean_Cardnum_30d_count_Ave,Merchnum_1d_count_Merchnum_3d_count_Ave,Merchnum_1d_mean_Merchnum_3d_count_Ave,Merchnum_1d_count_Merchnum_7d_count_Ave,Merchnum_1d_mean_Merchnum_7d_count_Ave,Merchnum_1d_count_Merchnum_14d_count_Ave,Merchnum_1d_mean_Merchnum_14d_count_Ave,Merchnum_1d_count_Merchnum_30d_count_Ave,Merchnum_1d_mean_Merchnum_30d_count_Ave
80770,5142110002,2010-10-12,9900020006406,X REVERSAL OF CR BAL REF,VA,22202.0,P,150.0,0,150.00,...,30.0,4500.00,1.0,150.000,0.170732,25.609756,0.237288,35.593220,0.146341,21.951220
89106,5142110081,2010-11-26,2094330000009,FRANKLIN COVEY COMPANY,UT,84119.0,P,636.2,0,636.20,...,30.0,19086.00,1.5,954.300,1.750000,1113.350000,1.555556,989.644444,1.250000,795.250000
16606,5142110081,2010-03-08,4503082412500,X OFFICE DEPOT #1,WA,98032.0,P,495.9,0,495.90,...,30.0,14877.00,3.0,1487.700,3.500000,1735.650000,4.666667,2314.200000,2.727273,1352.454545
16778,5142110081,2010-03-08,4503082412500,X FRANKLIN COVEY,WA,98032.0,P,636.2,0,566.05,...,30.0,8490.75,3.0,849.075,4.666667,1320.783333,7.000000,1981.175000,5.000000,1415.125000
95589,5142110081,2010-12-27,4503082412500,OFFICE DEPOT #1078,WA,98032.0,P,495.9,0,495.90,...,30.0,14877.00,3.0,1487.700,3.500000,1735.650000,2.000000,991.800000,3.000000,1487.700000


In [23]:
data_sorted.shape

(96366, 360)

In [24]:
data_sorted.to_csv('featureEngineering_yifeng.csv')